## Adventurework Försäljningsanalys
### Styfle med analysen:
Denna analys för att undersöka försäljningsdata från AdventureWorks-databasen
för att identifiera mönster inom produktutbud, försäljning över tid, kundbeteenden samt
regionella skillnader.
Målet med analysen är att ge affärsinsikter som kan stödja strategiska beslut gällande sortement, marknadsföring och kundfokus.

In [22]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus

pyodbc.drivers()

user = "    SA"
password = quote_plus("Sallamati2.")
server = 'localhost:1433'
database = 'AdventureWorks2025'
driver = quote_plus("ODBC Driver 18 for SQL server")


connection_string = (
    f"mssql+pyodbc://{user}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)


try:
    with engine.connect():
        print("Anslutning till SQL Server lyckades")
except Exception as e:
    print("Kunde inte ansluta", e)

Kunde inte ansluta (pyodbc.InterfaceError) ('28000', "[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '    SA'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '    SA'. (18456)")
(Background on this error at: https://sqlalche.me/e/20/rvf5)


In [23]:
def query_df(sql: str):
    with engine.connect() as conn:
        return pd.read_sql_query(text(sql), conn)

In [24]:
query_vis1 = """SELECT"""

df = pd.read_sql(query_vis1, engine)
plt.figure()
plt.bar(df["Category"], df["ProductCount"])
plt.title("Antal produkter per kategori")
plt.xlabel("Kategori")
plt.ylabel("Antal produkter")
plt.xticks(rotation=45)
plt.tight.layout()
plt.show()


InterfaceError: (pyodbc.InterfaceError) ('28000', "[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '    SA'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '    SA'. (18456)")
(Background on this error at: https://sqlalche.me/e/20/rvf5)